In [1]:
import os
import sys
src_path = sys.path[0].replace("scripts", "src")
#replace notebook as scripts
data_path = sys.path[0].replace("scripts", "data")
if src_path not in sys.path:
    sys.path.append(src_path)

out_path = sys.path[0].replace("scripts", "output")

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
import copy

from grid_manager import MPC_op
from data_loader import UCSD_dataloader
from battery_model import Battery_base
from optimizer import Battery_optimizer
from predictor import *
from exp_manager import ExperimentManager

# from ev_manager import EVmanager
# import copy

# exp based on XGBoost prediction added bat 3h

In [2]:
exp_suffix = "July2023_3"
exp_folder = os.path.join(out_path, "experiments", exp_suffix)
debug_folder = os.path.join(out_path, "debug_test")
assert os.path.exists(exp_folder)

In [3]:
bat_params_sample ={
    "bat_capacity": None,
    "bat_p_max": 3, # i.e., capacity (kWh) / p_bat_max (kW) = 3 (h)
    "bat_p_min": 3, # can omit, then p_bat_min = p_bat_max
    "bat_price": 150, # $/kWh (old: 1000, ref: Tesla Powerwall)
    "bat_efficacy": 0.98, 
    "bat_life_0": 3650, # days.
    "bat_cycle_0": 3000, # cycles in lifetime
    # battery degradation params
    "deg_model": "throughput",  
        # valid values: "throughput", "Crate", "rainflow", "DOD"
    #   [1. degradation ~ high C-rate]
    "deg_Crate_thres": (0.25, 0.25, 0.25, 0.25),
    "deg_Crate_lambda": (0.8, 1, 1.5, 2),
    #   [2. degradation ~ large cycle depth]
    "deg_rainflow_thres": (0.2, 0.2, 0.2, 0.4),
    "deg_rainflow_lambda": (0.6, 1, 1.5, 1.8),
    #   [3. degradation ~ low SoE range]
    "deg_DOD_thres": (0.5, 0.2, 0.2, 0.1),
    "deg_DOD_lambda": (1.3, 1.15, 0.85, 0.6),
}

op_params_sample = {"K": 96,
            "dc_price": 0.6,
            "ev_efficacy": 0.98,
            "energy_price_sell": 0.6, 
            "deg_model_opt": "unconscious",
            "ev_charge_rule": "flex",
            "ev_charge_rule_default": "unif",
            "p_grid_max": "1.5",}


In [4]:
class MPC_ExperimentManager(ExperimentManager):
    """
    Main variable: battery size (normalized to hr: to bld_load)
    """
    def run_one_trial(self, params, save_fn):
        
        # params: keys: "pred_model", "strategy", "B_kWh", "deg_model_opt", "deg_model", 
        #   "start", "end", "p_grid_max", "price_dc", "price_sell", "ev_charge_rule"

        op_params = copy.deepcopy(op_params_sample)
        bat_params = copy.deepcopy(bat_params_sample)

        bat_params["deg_model"] = params.get("deg_model", "DOD")
        bat_params["bat_capacity"] = params.get("B_kWh", 350)
        strategy = params.get("strategy", "optimal")
        op_params["deg_model_opt"] = params.get("deg_model_opt", "rainflow") 
        op_params["ev_charge_rule"] = params.get("ev_charge_rule", "flex")
        op_params["energy_price_sell"] = params.get("price_sell", "rainflow")
        op_params["dc_price"] = params.get("price_dc", 0.6) 
        p_grid_max = params.get("p_grid_max", "1.5") 
        op_params["p_grid_max"] = None if p_grid_max is None else str(p_grid_max)
        pred_model = params.get("pred_model", "GT") 
        
        # 2023/05/30 LunLong
        # Trying to add bld pv ev into grid search parmas
        bld=params.get("bld", "Hopkins")
        pv=params.get("pv", "Hopkins")
        ev=params.get("ev","OSLER")   
        
        # 2023/07/21 LunLong
        # Check params sanity with XGBoost prediction added
        #if params.get("pred_model")=='Prediction':
        #    if not params.get("bld")=="Sum" & params.get("pv")=='Sum':
        #        raise Exception("Incompatible pred_model and bld/pv setting.")
         
        if bld is None or pv is None or ev is None:
            raise Exception("Building related params are not prepoperly initiated")

        def convert_time(s):
            idx_hyphen = s.index("-")
            month, day = int(s[:idx_hyphen]), int(s[idx_hyphen+1:])
            return datetime(2019, month, day, 0, 0)
        t_start = convert_time(params.get("start", "10-1"))
        t_end = convert_time(params.get("end", "10-8"))

        mpc = MPC_op()

        # Step 1: load data
        mpc.load_data(loader=UCSD_dataloader, 
            tstart=datetime(2019,1,1,0,0), tend=datetime(2020,1,1,0,0), delta=0.25,
            bld=bld, pv=pv, ev=ev, pv_to_bld=0.5, ev_to_bld=0.25, Pmax=10)

        # Step 2: Load historical data
        mpc.init_historical_data(loader=UCSD_dataloader,
            tstart=datetime(2018,1,1,0,0), tend=datetime(2019,12,31,23,59), delta=0.25,
            bld=bld, pv=pv, ev=ev, pv_to_bld=0.5, ev_to_bld=0.25, Pmax=10)

        # Step 3: specify other operational params
        optimizer_params = {"strategy": strategy, "language":"gurobi"}

        mpc.init_op_params(optimizer=Battery_optimizer, optimizer_params=optimizer_params, delta_0=0.25, **op_params)

        # Step 4: specify battery
        mpc.init_battery(model=Battery_base, params=bat_params, delta_0=0.25)

        # Step 5: initialize predictor
        # [Yi, 2023/03/08] modify predictor def
        mpc.init_predictor(shortcut=pred_model)

        # Step 6: initialize save_config
        mpc.init_save_config(save_fn=save_fn[:-5],  # FIXME: remove ".xlsx"
            folder_path=self.save_path, log_pred_action=True,
            checkpoints="1D", recovery=True, recovery_from=None,
            )

        mpc.run(tstart=t_start, tend=t_end, exe_K=4, save=True)
        
        stats = dict(mpc.summary["All"])
        
        return stats

In [16]:
exp_suffix = "July2023_4"
exp_folder = os.path.join(out_path, "experiments", exp_suffix)
debug_folder = os.path.join(out_path, "debug_test")
assert os.path.exists(exp_folder)
log_fn = os.path.join(exp_folder, "LOG-MPC-MSC-3h-bat-Jul-Sep.xlsx")
save_path = os.path.join(exp_folder, "MPC-demo")
#save_path
log_fn

'd:\\Codes\\GIthub_repo\\Energy_grid\\output\\experiments\\July2023_4\\LOG-MPC-MSC-3h-bat-Jul-Sep.xlsx'

# MONTH BASED

In [17]:
for i in range(70):
    try:
        em = MPC_ExperimentManager(log_fn=log_fn, save_path=save_path, save=True, exp_prefix="MPC")
        var_keys = [
                    "method", "strategy",
                    "pred_model", "deg_model_opt", "p_grid_max", 
                    "price_dc", "price_sell", 
                    "ev_charge_rule", 
                    "B_kWh",  "deg_model", 
                    "start", "end", 
                    "bld", "ev", "pv"]
        em.run(keys=var_keys, num_trials=120)
    except Exception as e:
        print('Reason:', e)  
        print('Failed in '+str(i)+'nd trial')
        continue

!!!!!!!!!! EV SHORT DURATION !!!!!!!!!! || drop 119 sessions
!!!!!!!!!! EV SHORT DURATION !!!!!!!!!! || drop 237 sessions
========================= 2019-07-01 00:00:00 =========================
========================= 2019-07-01 01:00:00 =========================
========================= 2019-07-01 02:00:00 =========================
========================= 2019-07-01 03:00:00 =========================
========================= 2019-07-01 04:00:00 =========================
========================= 2019-07-01 05:00:00 =========================
========================= 2019-07-01 06:00:00 =========================
========================= 2019-07-01 07:00:00 =========================
========================= 2019-07-01 08:00:00 =========================
========================= 2019-07-01 09:00:00 =========================
========================= 2019-07-01 10:00:00 =========================
========================= 2019-07-01 11:00:00 =========================
==============

d:\Codes\GIthub_repo\Energy_grid\src\optimizer.py:1246: UserWarning: EV demand infeas: s=0, idx=[21, 22]
  warnings.warn("EV demand infeas: s={}, idx={}".format(s, infeas_idx))
d:\Codes\GIthub_repo\Energy_grid\src\optimizer.py:1254: RuntimeWarning: invalid value encountered in divide
  e_targ_cut = e_init + (e_targ - e_init) * (td_cut-ta)/(td-ta)


Reason: Constant is Nan
Failed in 0nd trial
!!!!!!!!!! EV SHORT DURATION !!!!!!!!!! || drop 119 sessions
!!!!!!!!!! EV SHORT DURATION !!!!!!!!!! || drop 237 sessions
========================= 2019-07-01 00:00:00 =========================
========================= 2019-07-01 01:00:00 =========================
========================= 2019-07-01 02:00:00 =========================
========================= 2019-07-01 03:00:00 =========================
========================= 2019-07-01 04:00:00 =========================
========================= 2019-07-01 05:00:00 =========================
========================= 2019-07-01 06:00:00 =========================
========================= 2019-07-01 07:00:00 =========================
========================= 2019-07-01 08:00:00 =========================
========================= 2019-07-01 09:00:00 =========================
========================= 2019-07-01 10:00:00 =========================
========================= 2019-07-01 11:00

d:\Codes\GIthub_repo\Energy_grid\src\optimizer.py:1246: UserWarning: EV demand infeas: s=0, idx=[21, 22]
  warnings.warn("EV demand infeas: s={}, idx={}".format(s, infeas_idx))
d:\Codes\GIthub_repo\Energy_grid\src\optimizer.py:1254: RuntimeWarning: invalid value encountered in divide
  e_targ_cut = e_init + (e_targ - e_init) * (td_cut-ta)/(td-ta)


Reason: Constant is Nan
Failed in 1nd trial
!!!!!!!!!! EV SHORT DURATION !!!!!!!!!! || drop 119 sessions
!!!!!!!!!! EV SHORT DURATION !!!!!!!!!! || drop 237 sessions
========================= 2019-07-01 00:00:00 =========================
========================= 2019-07-01 01:00:00 =========================
========================= 2019-07-01 02:00:00 =========================
========================= 2019-07-01 03:00:00 =========================
========================= 2019-07-01 04:00:00 =========================
========================= 2019-07-01 05:00:00 =========================
========================= 2019-07-01 06:00:00 =========================
========================= 2019-07-01 07:00:00 =========================
========================= 2019-07-01 08:00:00 =========================
========================= 2019-07-01 09:00:00 =========================
========================= 2019-07-01 10:00:00 =========================
========================= 2019-07-01 11:00

...

========================= 2019-08-13 10:00:00 =========================
========================= 2019-08-13 11:00:00 =========================
========================= 2019-08-13 12:00:00 =========================
========================= 2019-08-13 13:00:00 =========================
========================= 2019-08-13 14:00:00 =========================
========================= 2019-08-13 15:00:00 =========================
========================= 2019-08-13 16:00:00 =========================
========================= 2019-08-13 17:00:00 =========================
========================= 2019-08-13 18:00:00 =========================
========================= 2019-08-13 19:00:00 =========================
========================= 2019-08-13 20:00:00 =========================
========================= 2019-08-13 21:00:00 =========================
========================= 2019-08-13 22:00:00 =========================
========================= 2019-08-13 23:00:00 ==================

d:\Codes\GIthub_repo\Energy_grid\src\optimizer.py:1246: UserWarning: EV demand infeas: s=0, idx=[7, 8, 9]
  warnings.warn("EV demand infeas: s={}, idx={}".format(s, infeas_idx))


========================= 2019-08-13 10:00:00 =========================
========================= 2019-08-13 11:00:00 =========================
========================= 2019-08-13 12:00:00 =========================
========================= 2019-08-13 13:00:00 =========================
========================= 2019-08-13 14:00:00 =========================
========================= 2019-08-13 15:00:00 =========================
========================= 2019-08-13 16:00:00 =========================
========================= 2019-08-13 17:00:00 =========================
========================= 2019-08-13 18:00:00 =========================
========================= 2019-08-13 19:00:00 =========================
========================= 2019-08-13 20:00:00 =========================
========================= 2019-08-13 21:00:00 =========================
========================= 2019-08-13 22:00:00 =========================
========================= 2019-08-13 23:00:00 ==================

d:\Codes\GIthub_repo\Energy_grid\src\optimizer.py:1246: UserWarning: EV demand infeas: s=0, idx=[7, 8, 9]
  warnings.warn("EV demand infeas: s={}, idx={}".format(s, infeas_idx))


========================= 2019-08-13 10:00:00 =========================
========================= 2019-08-13 11:00:00 =========================
========================= 2019-08-13 12:00:00 =========================
========================= 2019-08-13 13:00:00 =========================
========================= 2019-08-13 14:00:00 =========================
========================= 2019-08-13 15:00:00 =========================
========================= 2019-08-13 16:00:00 =========================
========================= 2019-08-13 17:00:00 =========================
========================= 2019-08-13 18:00:00 =========================
========================= 2019-08-13 19:00:00 =========================
========================= 2019-08-13 20:00:00 =========================
========================= 2019-08-13 21:00:00 =========================
========================= 2019-08-13 22:00:00 =========================
========================= 2019-08-13 23:00:00 ==================

# WEEK BASED

In [6]:
for i in range(70):
    try:
        em = MPC_ExperimentManager(log_fn=log_fn, save_path=save_path, save=True, exp_prefix="MPC")
        var_keys = [
                    "method", "strategy",
                    "pred_model", "deg_model_opt", "p_grid_max", 
                    "price_dc", "price_sell", 
                    "ev_charge_rule", 
                    "B_kWh",  "deg_model", 
                    "start", "end", 
                    "bld", "ev", "pv"]
        em.run(keys=var_keys, num_trials=120)
    except Exception as e:
        print('Reason:', e)  
        print('Failed in '+str(i)+'nd trial')
        continue

!!!!!!!!!! EV SHORT DURATION !!!!!!!!!! || drop 119 sessions
!!!!!!!!!! EV SHORT DURATION !!!!!!!!!! || drop 237 sessions
========================= 2019-07-01 00:00:00 =========================
========================= 2019-07-01 01:00:00 =========================
========================= 2019-07-01 02:00:00 =========================
========================= 2019-07-01 03:00:00 =========================
========================= 2019-07-01 04:00:00 =========================
========================= 2019-07-01 05:00:00 =========================
========================= 2019-07-01 06:00:00 =========================
========================= 2019-07-01 07:00:00 =========================
========================= 2019-07-01 08:00:00 =========================
========================= 2019-07-01 09:00:00 =========================
========================= 2019-07-01 10:00:00 =========================
========================= 2019-07-01 11:00:00 =========================
==============

d:\Codes\GIthub_repo\Energy_grid\src\optimizer.py:1246: UserWarning: EV demand infeas: s=0, idx=[21, 22]
  warnings.warn("EV demand infeas: s={}, idx={}".format(s, infeas_idx))
d:\Codes\GIthub_repo\Energy_grid\src\optimizer.py:1254: RuntimeWarning: invalid value encountered in divide
  e_targ_cut = e_init + (e_targ - e_init) * (td_cut-ta)/(td-ta)


Reason: Constant is Nan
Failed in 0nd trial
!!!!!!!!!! EV SHORT DURATION !!!!!!!!!! || drop 119 sessions
!!!!!!!!!! EV SHORT DURATION !!!!!!!!!! || drop 237 sessions
========================= 2019-07-22 00:00:00 =========================
========================= 2019-07-22 01:00:00 =========================
========================= 2019-07-22 02:00:00 =========================
========================= 2019-07-22 03:00:00 =========================
========================= 2019-07-22 04:00:00 =========================
========================= 2019-07-22 05:00:00 =========================
========================= 2019-07-22 06:00:00 =========================
========================= 2019-07-22 07:00:00 =========================
========================= 2019-07-22 08:00:00 =========================
========================= 2019-07-22 09:00:00 =========================
========================= 2019-07-22 10:00:00 =========================
========================= 2019-07-22 11:00

d:\Codes\GIthub_repo\Energy_grid\src\optimizer.py:1246: UserWarning: EV demand infeas: s=0, idx=[21, 22]
  warnings.warn("EV demand infeas: s={}, idx={}".format(s, infeas_idx))
d:\Codes\GIthub_repo\Energy_grid\src\optimizer.py:1254: RuntimeWarning: invalid value encountered in divide
  e_targ_cut = e_init + (e_targ - e_init) * (td_cut-ta)/(td-ta)


Reason: Constant is Nan
Failed in 1nd trial
!!!!!!!!!! EV SHORT DURATION !!!!!!!!!! || drop 119 sessions
!!!!!!!!!! EV SHORT DURATION !!!!!!!!!! || drop 237 sessions
========================= 2019-07-22 00:00:00 =========================
========================= 2019-07-22 01:00:00 =========================
========================= 2019-07-22 02:00:00 =========================
========================= 2019-07-22 03:00:00 =========================
========================= 2019-07-22 04:00:00 =========================
========================= 2019-07-22 05:00:00 =========================
========================= 2019-07-22 06:00:00 =========================
========================= 2019-07-22 07:00:00 =========================
========================= 2019-07-22 08:00:00 =========================
========================= 2019-07-22 09:00:00 =========================
========================= 2019-07-22 10:00:00 =========================
========================= 2019-07-22 11:00

...

========================= 2019-10-02 18:00:00 =========================
========================= 2019-10-02 19:00:00 =========================
========================= 2019-10-02 20:00:00 =========================
========================= 2019-10-02 21:00:00 =========================
========================= 2019-10-02 22:00:00 =========================
========================= 2019-10-02 23:00:00 =========================
========================= 2019-10-03 00:00:00 =========================
========================= 2019-10-03 01:00:00 =========================
========================= 2019-10-03 02:00:00 =========================
========================= 2019-10-03 03:00:00 =========================
========================= 2019-10-03 04:00:00 =========================
========================= 2019-10-03 05:00:00 =========================
========================= 2019-10-03 06:00:00 =========================
========================= 2019-10-03 07:00:00 ==================

d:\Codes\GIthub_repo\Energy_grid\src\optimizer.py:1246: UserWarning: EV demand infeas: s=0, idx=[13, 14]
  warnings.warn("EV demand infeas: s={}, idx={}".format(s, infeas_idx))


========================= 2019-10-02 18:00:00 =========================
========================= 2019-10-02 19:00:00 =========================
========================= 2019-10-02 20:00:00 =========================
========================= 2019-10-02 21:00:00 =========================
========================= 2019-10-02 22:00:00 =========================
========================= 2019-10-02 23:00:00 =========================
========================= 2019-10-03 00:00:00 =========================
========================= 2019-10-03 01:00:00 =========================
========================= 2019-10-03 02:00:00 =========================
========================= 2019-10-03 03:00:00 =========================
========================= 2019-10-03 04:00:00 =========================
========================= 2019-10-03 05:00:00 =========================
========================= 2019-10-03 06:00:00 =========================
========================= 2019-10-03 07:00:00 ==================

d:\Codes\GIthub_repo\Energy_grid\src\optimizer.py:1246: UserWarning: EV demand infeas: s=0, idx=[13, 14]
  warnings.warn("EV demand infeas: s={}, idx={}".format(s, infeas_idx))


========================= 2019-10-02 20:00:00 =========================
========================= 2019-10-02 21:00:00 =========================
========================= 2019-10-02 22:00:00 =========================
========================= 2019-10-02 23:00:00 =========================
========================= 2019-10-03 00:00:00 =========================
========================= 2019-10-03 01:00:00 =========================
========================= 2019-10-03 02:00:00 =========================
========================= 2019-10-03 03:00:00 =========================
========================= 2019-10-03 04:00:00 =========================
========================= 2019-10-03 05:00:00 =========================
========================= 2019-10-03 06:00:00 =========================
========================= 2019-10-03 07:00:00 =========================
========================= 2019-10-03 08:00:00 =========================
========================= 2019-10-03 09:00:00 ==================

# week based all ev

In [6]:
exp_suffix = "July2023_5"
exp_folder = os.path.join(out_path, "experiments", exp_suffix)
debug_folder = os.path.join(out_path, "debug_test")
assert os.path.exists(exp_folder)

log_fn = os.path.join(exp_folder, "LOG-MPC-MSC-3h-bat-Jul-Sep.xlsx")
save_path = os.path.join(exp_folder, "MPC-demo")

for i in range(70):

    try:
        em = MPC_ExperimentManager(log_fn=log_fn, save_path=save_path, save=True, exp_prefix="MPC")
        var_keys = [
                    "method", "strategy",
                    "pred_model", "deg_model_opt", "p_grid_max", 
                    "price_dc", "price_sell", 
                    "ev_charge_rule", 
                    "B_kWh",  "deg_model", 
                    "start", "end", 
                    "bld", "ev", "pv"]
        em.run(keys=var_keys, num_trials=120)
    except Exception as e:
        print('Reason:', e)  
        print('Failed in '+str(i)+'nd trial')
        continue

!!!!!!!!!! EV SHORT DURATION !!!!!!!!!! || drop 106 sessions
!!!!!!!!!! EV SHORT DURATION !!!!!!!!!! || drop 225 sessions
========================= 2019-09-30 00:00:00 =========================
========================= 2019-09-30 01:00:00 =========================
========================= 2019-09-30 02:00:00 =========================
========================= 2019-09-30 03:00:00 =========================
========================= 2019-09-30 04:00:00 =========================
========================= 2019-09-30 05:00:00 =========================
========================= 2019-09-30 06:00:00 =========================
========================= 2019-09-30 07:00:00 =========================


d:\Codes\GIthub_repo\Energy_grid\src\optimizer.py:1246: UserWarning: EV demand infeas: s=0, idx=[1]
  warnings.warn("EV demand infeas: s={}, idx={}".format(s, infeas_idx))


========================= 2019-09-30 08:00:00 =========================
========================= 2019-09-30 09:00:00 =========================


d:\Codes\GIthub_repo\Energy_grid\src\optimizer.py:1246: UserWarning: EV demand infeas: s=0, idx=[22]
  warnings.warn("EV demand infeas: s={}, idx={}".format(s, infeas_idx))


========================= 2019-09-30 10:00:00 =========================
========================= 2019-09-30 11:00:00 =========================
========================= 2019-09-30 12:00:00 =========================
========================= 2019-09-30 13:00:00 =========================
========================= 2019-09-30 14:00:00 =========================
========================= 2019-09-30 15:00:00 =========================
========================= 2019-09-30 16:00:00 =========================
========================= 2019-09-30 17:00:00 =========================
========================= 2019-09-30 18:00:00 =========================
========================= 2019-09-30 19:00:00 =========================


...

========================= 2019-10-06 08:00:00 =========================
========================= 2019-10-06 09:00:00 =========================
========================= 2019-10-06 10:00:00 =========================
========================= 2019-10-06 11:00:00 =========================
========================= 2019-10-06 12:00:00 =========================
========================= 2019-10-06 13:00:00 =========================
========================= 2019-10-06 14:00:00 =========================


d:\Codes\GIthub_repo\Energy_grid\src\optimizer.py:1246: UserWarning: EV demand infeas: s=0, idx=[3]
  warnings.warn("EV demand infeas: s={}, idx={}".format(s, infeas_idx))


========================= 2019-10-06 15:00:00 =========================


d:\Codes\GIthub_repo\Energy_grid\src\optimizer.py:1246: UserWarning: EV demand infeas: s=0, idx=[3]
  warnings.warn("EV demand infeas: s={}, idx={}".format(s, infeas_idx))


========================= 2019-10-06 16:00:00 =========================
========================= 2019-10-06 17:00:00 =========================
========================= 2019-10-06 18:00:00 =========================
========================= 2019-10-06 19:00:00 =========================
========================= 2019-10-06 20:00:00 =========================
========================= 2019-10-06 21:00:00 =========================
========================= 2019-10-06 22:00:00 =========================
========================= 2019-10-06 23:00:00 =========================
========================= FINISH =========================
Done, trial 70
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
